**Imports**

In [1]:
import axelrod as axl

In [2]:
import itertools

import numpy as np

In [3]:
# importing the bayesian code like this for now

from importlib.machinery import SourceFileLoader

bayesian = SourceFileLoader("bayesian", "src/bayesian.py").load_module()

### A few details about `axelrod`

We can use `axelrod` to simulate the interaction of a given memory one strategy and of a sequence.

In [4]:
from axelrod.action import Action

C, D = Action.C, Action.D

Now the variables `C` and `D` correspond to the actions `C` and `D`.

This is how we can define a memory-one player and their initial move.

In [5]:
q = axl.MemoryOnePlayer((1, 1, 1, 1), initial=D)
q

Generic Memory One Player: (1, 1, 1, 1), D

For the sequence we can use the class `axl.Cycler` which takes an input an `str` sequence. If the sequence is smaller than the number of turns the Cycler just keeps cycling over the same sequence.

In [6]:
p = axl.Cycler("CCD")

In [7]:
match = axl.Match(players=[p, q], turns=3)

_ = match.play()

In [8]:
match.result

[(C, D), (C, C), (D, C)]

In [10]:
match.scores()

[(0, 5), (3, 3), (5, 0)]

In [12]:
match.final_score()

(8, 8)

In [13]:
match.final_score_per_turn()

(2.6666666666666665, 2.6666666666666665)

We can have a match with errors as well:

In [14]:
match = axl.Match(players=[p, q], turns=3, noise=0.1, seed=10)

_ = match.play()

In [15]:
match.result

[(C, C), (C, C), (D, C)]

### Inferring strategies

Let's assume we are trying to fit the 16 pure strategies.

1. Initially we use `axelrod` for getting the co-player's actions.

In [49]:
p = axl.Cycler("CCDCDDDCDC")

In [50]:
match = axl.Match(players=[p, q], turns=10)

_ = match.play()

In [51]:
coplayers_actions = [history[1] for history in match.result]

coplayers_actions

[D, C, C, C, C, C, C, C, C, C]

In [52]:
game_history = match.result

game_history

[(C, D),
 (C, C),
 (D, C),
 (C, C),
 (D, C),
 (D, C),
 (D, C),
 (C, C),
 (D, C),
 (C, C)]

In [53]:
num_possible_s = 32

In [54]:
last_turn_outcomes = ["p0"] + list(itertools.product([C, D], repeat=2))

In [55]:
pure_transitions = list(itertools.product([C, D], repeat=5))

In [56]:
pure_strategies = {f"M{i}": 
               {k:v for k, v in zip(last_turn_outcomes, transitions)} 
               for i, transitions in enumerate(pure_transitions)}

In [57]:
pure_strategies

{'M0': {'p0': C, (C, C): C, (C, D): C, (D, C): C, (D, D): C},
 'M1': {'p0': C, (C, C): C, (C, D): C, (D, C): C, (D, D): D},
 'M2': {'p0': C, (C, C): C, (C, D): C, (D, C): D, (D, D): C},
 'M3': {'p0': C, (C, C): C, (C, D): C, (D, C): D, (D, D): D},
 'M4': {'p0': C, (C, C): C, (C, D): D, (D, C): C, (D, D): C},
 'M5': {'p0': C, (C, C): C, (C, D): D, (D, C): C, (D, D): D},
 'M6': {'p0': C, (C, C): C, (C, D): D, (D, C): D, (D, D): C},
 'M7': {'p0': C, (C, C): C, (C, D): D, (D, C): D, (D, D): D},
 'M8': {'p0': C, (C, C): D, (C, D): C, (D, C): C, (D, D): C},
 'M9': {'p0': C, (C, C): D, (C, D): C, (D, C): C, (D, D): D},
 'M10': {'p0': C, (C, C): D, (C, D): C, (D, C): D, (D, D): C},
 'M11': {'p0': C, (C, C): D, (C, D): C, (D, C): D, (D, D): D},
 'M12': {'p0': C, (C, C): D, (C, D): D, (D, C): C, (D, D): C},
 'M13': {'p0': C, (C, C): D, (C, D): D, (D, C): C, (D, D): D},
 'M14': {'p0': C, (C, C): D, (C, D): D, (D, C): D, (D, D): C},
 'M15': {'p0': C, (C, C): D, (C, D): D, (D, C): D, (D, D): D},
 '

2. We use the `MemoryOne` class we have defined in `bayesian` which gives us the probability of the action $i$ happening after the history $h_t$.

In [58]:
strategies_to_fit = [bayesian.MemoryOne(error=0.001, states_action_dict=value)
                     for value in pure_strategies.values()]

In [67]:
priors = [bayesian.init_prior_uniform(num_possible_s) for _ in range(num_possible_s)]


# Opening Move

opening_move = coplayers_actions[0]


likelihoods = [strategy.likelihood(opening_move, "p0") for strategy in strategies_to_fit]

posteriors = [bayesian.posterior(myprior,
                                 mylikelihood, 
                                 priors,
                                 likelihoods) for myprior, mylikelihood in zip(priors, likelihoods)]

priors = posteriors 
    
# The rest

for turn, turn_action in enumerate(coplayers_actions[1:]):

    likelihoods = [strategy.likelihood(turn_action,
                                       game_history[turn]) for strategy in strategies_to_fit]

    posteriors = [bayesian.posterior(myprior,
                                     mylikelihood, 
                                     priors,
                                     likelihoods) for myprior, mylikelihood in zip(priors, likelihoods)]

    priors = posteriors

In [68]:
np.array(posteriors).round(3)

array([0.008, 0.008, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.326, 0.326,
       0.   , 0.   , 0.007, 0.007, 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   ])

In [69]:
np.argmax(np.array(posteriors).round(3))

16